In [12]:
from upscaler.data import load_images_from_dir

from keras.models import load_model
import tensorflow as tf
from tqdm import tqdm_notebook

In [14]:
load_images_from_dir(
        '../images/ukiyo-e_fullhd',
        '.jpg',
        limit = 10000,
        prog_func = tqdm_notebook
    )

,filename,image_size,image_hr
0,0017-courting-komachi-1792.jpg,"(1920, 1080)",<PIL.Image.Image image mode=RGB size=1920x1080...
1,0019-portrait-of-naniwaya-okita-1793.jpg,"(1920, 1080)",<PIL.Image.Image image mode=RGB size=1920x1080...
2,0021-from-the-series-kasen-koi-no-bu-1794.jpg,"(1920, 1080)",<PIL.Image.Image image mode=RGB size=1920x1080...
3,0022-hour-of-the-snake-1794.jpg,"(1920, 1080)",<PIL.Image.Image image mode=RGB size=1920x1080...
4,0024-oiran-hanaogi.jpg,"(1920, 1080)",<PIL.Image.Image image mode=RGB size=1920x1080...
5,0025-sharaku-1794-segawa-tomisabur-as-yadorigi...,"(1920, 1080)",<PIL.Image.Image image mode=RGB size=1920x1080...
6,0026-arashi-ry-z-i-as-ishibe-kinkichi.jpg,"(1920, 1080)",<PIL.Image.Image image mode=RGB size=1920x1080...
7,0028-met-dp130180.jpg,"(1920, 1080)",<PIL.Image.Image image mode=RGB size=1920x1080...
8,0030-iwai-hanshir-iv-as-the-wet-nurse-shigenoi...,"(1920, 1080)",<PIL.Image.Image image mode=RGB size=1920x1080...
9,0033-met-dp119557.jpg,"(1920, 1080)",<PIL.Image.Image image mode=RGB size=1920x1080...


In [5]:
upscaler = load_model('../example_movie_frames/model_upscaler_mini_resnet-att_vgg-only_x4_best.h5', custom_objects = { "tf": tf })

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [11]:
upscaler.layers[0].input

<tf.Tensor 'initial/input_1:0' shape=(?, 128, 128, 3) dtype=float32>